In [1]:
import pandas as pd
import sys
import json
from google.colab import drive
import re
# split the data into train and validation
from sklearn.model_selection import train_test_split
import warnings
from collections import Counter
warnings.filterwarnings("ignore")
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
path = "/content/drive/MyDrive/COVID19 Fake News Detection in English/"
bert_path = path + "bert-large-uncased/output/"
covid_bert_path = path + "covid-twitter-bert/output/"
roberta_path = path + "roberta-large/output/"

bert_df = pd.read_csv(f"{bert_path}bert_large_two_train_16_2e5.csv")
roberta_df = pd.read_csv(f"{roberta_path}roberta_large_four_train_16_2e5.csv")
biobert_df = pd.read_csv(f"{covid_bert_path}bio_bert_two_train_16_3e5.csv")
covid_bert_df = pd.read_csv(f"{covid_bert_path}covid_bert_two_train_16_3e5.csv")
sci_bert_df = pd.read_csv(f"{covid_bert_path}sci_bert_two_train_32_3e5.csv")
sci_bert_df.tail()

,id,tweet,y_pred
2135,2136,#CoronaVirusUpdates: State-wise details of Tot...,0.0
2136,2137,Tonight 12(midnight) onwards Disaster Manageme...,1.0
2137,2138,296 new cases of #COVID19Nigeria; Plateau-85 E...,0.0
2138,2139,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...,0.0
2139,2140,More than half of pregnant women recently admi...,1.0


In [31]:
index = []
target = []
bert_pred  = []
roberta_pred = []
biobert_pred = []
covid_bert_pred = []
sci_bert_pred = []
for _id in range(len(bert_df)):
  index.append(bert_df['id'][_id])  
  bert_pred.append(bert_df['y_pred'][_id])
  roberta_pred.append(roberta_df['y_pred'][_id])
  biobert_pred.append(biobert_df['y_pred'][_id])
  covid_bert_pred.append(covid_bert_df['y_pred'][_id])
  sci_bert_pred.append(sci_bert_df['y_pred'][_id])

```
Bert large Uncased ************
last two hidden bert_large train batch 16  lr = 2e-5
F1 Acc: 0.967
```

```
last two hidden covid bert train batch 16 lr = 3e5 ************
F1 Acc: 0.98
```

```
last two hidden bio bert train batch 16 lr = 3e5
F1 Acc: 0.964
```

```
last two hidden sci bert train batch 32 lr = 3e5 [need to run more than 4 epochs]
F1 Acc:  0.958
```

```
last four roberta large train batch 16 lr 2e5 *************
accuracy 0.973
```



In [32]:
mx_vote_df = pd.DataFrame()
mx_vote_df['id'] = index
mx_vote_df['bert'] = bert_pred
mx_vote_df['roberta'] = roberta_pred
#mx_vote_df['biobert'] = biobert_pred
mx_vote_df['covid_bert'] = covid_bert_pred
#mx_vote_df['sci_bert'] = sci_bert_pred

In [33]:
mx_vote_df

,id,bert,roberta,covid_bert
0,1,0.0,0.0,0.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,0.0,0.0,0.0
4,5,0.0,0.0,0.0
...,...,...,...,...
2135,2136,0.0,0.0,0.0
2136,2137,1.0,1.0,1.0
2137,2138,0.0,0.0,0.0
2138,2139,0.0,0.0,0.0


In [34]:
print("bert = \n", bert_df.y_pred.value_counts())
print("roberta = \n", roberta_df.y_pred.value_counts())
#print("bio-bert = \n", biobert_df.y_pred.value_counts())
print("covid_bert = \n", covid_bert_df.y_pred.value_counts())
#print("sci-bert = \n", sci_bert_df.y_pred.value_counts())

bert = 
 0.0    1132
1.0    1008
Name: y_pred, dtype: int64
roberta = 
 0.0    1098
1.0    1042
Name: y_pred, dtype: int64
covid_bert = 
 0.0    1124
1.0    1016
Name: y_pred, dtype: int64


In [35]:
print("voting bert = \n", mx_vote_df.bert.value_counts())
print("voting roberta = \n", mx_vote_df.roberta.value_counts())
#print("voting bio-bert = \n", mx_vote_df.biobert.value_counts())
print("voting covid_bert = \n",mx_vote_df.covid_bert.value_counts())
#print("voting sci-bert = \n", mx_vote_df.sci_bert.value_counts())

voting bert = 
 0.0    1132
1.0    1008
Name: bert, dtype: int64
voting roberta = 
 0.0    1098
1.0    1042
Name: roberta, dtype: int64
voting covid_bert = 
 0.0    1124
1.0    1016
Name: covid_bert, dtype: int64


In [36]:
preds = []
for index in range(len(mx_vote_df)):
  values = mx_vote_df.iloc[index].values[1:]
  sorted_x = sorted(Counter(values).items(), key=lambda kv: kv[1],reverse=True)
  preds.append(sorted_x[0][0])
mx_vote_df['preds'] = preds

In [37]:
mx_vote_df

,id,bert,roberta,covid_bert,preds
0,1,0.0,0.0,0.0,0.0
1,2,1.0,1.0,1.0,1.0
2,3,1.0,1.0,1.0,1.0
3,4,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0
...,...,...,...,...,...
2135,2136,0.0,0.0,0.0,0.0
2136,2137,1.0,1.0,1.0,1.0
2137,2138,0.0,0.0,0.0,0.0
2138,2139,0.0,0.0,0.0,0.0


In [38]:
print("voting bert = \n", mx_vote_df.bert.value_counts())
print("voting roberta = \n", mx_vote_df.roberta.value_counts())
print("voting covid_bert = \n",mx_vote_df.covid_bert.value_counts())
print("voting prediction count = \n", mx_vote_df.preds.value_counts())

voting bert = 
 0.0    1132
1.0    1008
Name: bert, dtype: int64
voting roberta = 
 0.0    1098
1.0    1042
Name: roberta, dtype: int64
voting covid_bert = 
 0.0    1124
1.0    1016
Name: covid_bert, dtype: int64
voting prediction count = 
 0.0    1116
1.0    1024
Name: preds, dtype: int64


In [42]:
path = '/content/drive/MyDrive/COVID19 Fake News Detection in English/input/'
sub_df = mx_vote_df[['id','preds']]
for i in range(len(sub_df)):
  if sub_df['preds'][i] == 1:
    sub_df.at[i,'label'] =  'fake'
  else:
    sub_df.at[i,'label'] =  'real'
sub_df = sub_df[['id','label']]
new_path = path+'answer.txt'
sub_df.to_csv(new_path, index=None, sep=',', mode='a')

In [43]:
sub_df.label.value_counts()

real    1116
fake    1024
Name: label, dtype: int64